In [17]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm

import random
import numpy as np
import pandas as pd

import pickle
import datetime as dt
import time

import itertools
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

import cv2

import keras

from keras.callbacks import TensorBoard

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from mpl_toolkits.axes_grid1 import ImageGrid

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.models import Sequential
from keras.models import Model


from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers import Dense, GlobalAveragePooling2D
from keras import optimizers
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau


from keras import backend as K

from keras.applications import vgg16, resnet50, mobilenet
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3

from keras.applications.inception_v3 import InceptionV3

from keras.applications.vgg16 import preprocess_input, decode_predictions

from keras.optimizers import SGD




np.random.seed(2)
sns.set(style='white', context='notebook', palette='deep')

In [18]:
def show_pred(preds, Y, val_breed, index, seq, ran):
    leng = len(preds)
    if seq:
        for i in range(index):
            if ran:
                index = random.randint(0, leng) 
            _, imagenet_class_name, prob = decode_predictions(preds, top=1)[index][0]
            plt.title("Original: " + val_breed[Y[index]] + "\nPrediction: " + imagenet_class_name)
            plt.imshow(X_train[index])
            plt.show()
    else:
            _, imagenet_class_name, prob = decode_predictions(preds, top=1)[index][0]
            plt.title("Original: " + val_breed[Y[index]] + "\nPrediction: " + imagenet_class_name)
            plt.imshow(X_train[index])
            plt.show()
        
def accuracy_func(preds, Y, val_breed):
    leng = len(preds)
    count = 0;
    for i in range(leng):
        _, imagenet_class_name, prob = decode_predictions(preds, top=1)[i][0]
        if val_breed[Y[i]] == imagenet_class_name:
            count+=1
    accuracy = (count/leng)*100
    
    print("Accuracy: ", accuracy)
    return accuracy

In [19]:
cache_dir = expanduser(join('~', '.keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)

In [20]:
#Load the VGG model
vgg_model = vgg16.VGG16(weights='imagenet')
 
#Load the Inception_V3 model
inception_model = inception_v3.InceptionV3(weights='imagenet')
 
#Load the ResNet50 model
resnet_model = resnet50.ResNet50(weights='imagenet')
 
#Load the MobileNet model
mobilenet_model = mobilenet.MobileNet(weights='imagenet')

In [21]:
training_path = 'data/pre/train'
validation_path = 'data/pre/validation'
testing_path = 'data/pre/test'
batch_size = 32
target_size=(224, 224)
norm = 255.0
class_mode='categorical'

In [22]:
train_datagen = ImageDataGenerator(
        rescale=1./norm,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./norm)
test_datagen = ImageDataGenerator(rescale=1./norm)

train_generator = train_datagen.flow_from_directory(
        training_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

test_generator = test_datagen.flow_from_directory(
        testing_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

Found 2657 images belonging to 120 classes.
Found 825 images belonging to 120 classes.
Found 874 images belonging to 120 classes.


In [23]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

In [24]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(120, activation='softmax')(x)


In [25]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [26]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True



0 input_10
1 conv2d_283
2 batch_normalization_283
3 activation_381
4 conv2d_284
5 batch_normalization_284
6 activation_382
7 conv2d_285
8 batch_normalization_285
9 activation_383
10 max_pooling2d_15
11 conv2d_286
12 batch_normalization_286
13 activation_384
14 conv2d_287
15 batch_normalization_287
16 activation_385
17 max_pooling2d_16
18 conv2d_291
19 batch_normalization_291
20 activation_389
21 conv2d_289
22 conv2d_292
23 batch_normalization_289
24 batch_normalization_292
25 activation_387
26 activation_390
27 average_pooling2d_28
28 conv2d_288
29 conv2d_290
30 conv2d_293
31 conv2d_294
32 batch_normalization_288
33 batch_normalization_290
34 batch_normalization_293
35 batch_normalization_294
36 activation_386
37 activation_388
38 activation_391
39 activation_392
40 mixed0
41 conv2d_298
42 batch_normalization_298
43 activation_396
44 conv2d_296
45 conv2d_299
46 batch_normalization_296
47 batch_normalization_299
48 activation_394
49 activation_397
50 average_pooling2d_29
51 conv2d_295
5

In [27]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=["accuracy"])

In [28]:
model_dir = 'output/models/'
log_file = "output/logs"

model_file = model_dir+"weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"

In [29]:
checkpoint = keras.callbacks.ModelCheckpoint(model_file, monitor='val_acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

In [30]:
tensorboard = keras.callbacks.TensorBoard(log_dir=log_file, histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)
tensorboard.set_model(model) 

In [31]:
callbacks_list = [checkpoint, tensorboard]

In [32]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit_generator(
    train_generator,
    steps_per_epoch=83,
    epochs=200,
    validation_data=validation_generator,
    validation_steps=25,
    verbose=1,
    callbacks=callbacks_list)

Epoch 1/200
83/83 [==============================] - ETA: 21:12 - loss: 4.9532 - acc: 0.0000e+ - ETA: 17:36 - loss: 4.9285 - acc: 0.0000e+ - ETA: 16:18 - loss: 4.9502 - acc: 0.0000e+ - ETA: 15:28 - loss: 4.9326 - acc: 0.0000e+ - ETA: 14:58 - loss: 4.9423 - acc: 0.0000e+ - ETA: 14:36 - loss: 4.9333 - acc: 0.0000e+ - ETA: 14:17 - loss: 4.9302 - acc: 0.0045   - ETA: 14:00 - loss: 4.9209 - acc: 0.00 - ETA: 13:44 - loss: 4.9196 - acc: 0.00 - ETA: 13:29 - loss: 4.9162 - acc: 0.00 - ETA: 13:15 - loss: 4.9130 - acc: 0.00 - ETA: 13:02 - loss: 4.9086 - acc: 0.00 - ETA: 12:48 - loss: 4.9139 - acc: 0.00 - ETA: 12:34 - loss: 4.9100 - acc: 0.01 - ETA: 12:22 - loss: 4.8997 - acc: 0.01 - ETA: 12:09 - loss: 4.8948 - acc: 0.01 - ETA: 11:57 - loss: 4.8912 - acc: 0.01 - ETA: 11:46 - loss: 4.8976 - acc: 0.01 - ETA: 11:33 - loss: 4.8941 - acc: 0.01 - ETA: 11:21 - loss: 4.8920 - acc: 0.01 - ETA: 11:10 - loss: 4.8965 - acc: 0.01 - ETA: 10:58 - loss: 4.8928 - acc: 0.01 - ETA: 10:47 - loss: 4.8936 - acc: 0.01 -

83/83 [==============================] - ETA: 16:14 - loss: 4.8090 - acc: 0.0000e+ - ETA: 16:43 - loss: 4.7280 - acc: 0.0156   - ETA: 16:13 - loss: 4.7689 - acc: 0.02 - ETA: 15:46 - loss: 4.7586 - acc: 0.02 - ETA: 15:17 - loss: 4.7697 - acc: 0.02 - ETA: 15:02 - loss: 4.7431 - acc: 0.02 - ETA: 14:43 - loss: 4.7319 - acc: 0.01 - ETA: 14:25 - loss: 4.7296 - acc: 0.01 - ETA: 14:13 - loss: 4.7284 - acc: 0.02 - ETA: 13:59 - loss: 4.7237 - acc: 0.02 - ETA: 13:49 - loss: 4.7190 - acc: 0.02 - ETA: 13:37 - loss: 4.7134 - acc: 0.02 - ETA: 13:26 - loss: 4.7170 - acc: 0.02 - ETA: 13:14 - loss: 4.7197 - acc: 0.02 - ETA: 13:01 - loss: 4.7208 - acc: 0.02 - ETA: 12:48 - loss: 4.7188 - acc: 0.02 - ETA: 12:39 - loss: 4.7210 - acc: 0.02 - ETA: 12:28 - loss: 4.7243 - acc: 0.02 - ETA: 12:17 - loss: 4.7226 - acc: 0.02 - ETA: 12:06 - loss: 4.7283 - acc: 0.02 - ETA: 11:54 - loss: 4.7324 - acc: 0.01 - ETA: 11:42 - loss: 4.7330 - acc: 0.02 - ETA: 11:30 - loss: 4.7381 - acc: 0.02 - ETA: 11:18 - loss: 4.7419 - acc

83/83 [==============================] - ETA: 14:50 - loss: 4.6587 - acc: 0.06 - ETA: 14:41 - loss: 4.6010 - acc: 0.04 - ETA: 14:23 - loss: 4.6119 - acc: 0.04 - ETA: 14:42 - loss: 4.6162 - acc: 0.04 - ETA: 14:24 - loss: 4.6183 - acc: 0.04 - ETA: 14:13 - loss: 4.6053 - acc: 0.04 - ETA: 14:08 - loss: 4.6171 - acc: 0.05 - ETA: 13:57 - loss: 4.5974 - acc: 0.05 - ETA: 13:43 - loss: 4.5973 - acc: 0.05 - ETA: 13:31 - loss: 4.6041 - acc: 0.04 - ETA: 13:20 - loss: 4.5947 - acc: 0.04 - ETA: 13:08 - loss: 4.5964 - acc: 0.04 - ETA: 12:55 - loss: 4.5921 - acc: 0.05 - ETA: 12:43 - loss: 4.5926 - acc: 0.05 - ETA: 12:31 - loss: 4.5824 - acc: 0.05 - ETA: 12:18 - loss: 4.5852 - acc: 0.06 - ETA: 12:06 - loss: 4.5839 - acc: 0.06 - ETA: 11:53 - loss: 4.5853 - acc: 0.06 - ETA: 11:41 - loss: 4.5887 - acc: 0.06 - ETA: 11:28 - loss: 4.5894 - acc: 0.05 - ETA: 11:16 - loss: 4.5835 - acc: 0.06 - ETA: 11:04 - loss: 4.5857 - acc: 0.06 - ETA: 10:53 - loss: 4.5892 - acc: 0.05 - ETA: 10:42 - loss: 4.5884 - acc: 0.05 -

83/83 [==============================] - ETA: 13:59 - loss: 4.3666 - acc: 0.12 - ETA: 13:32 - loss: 4.3997 - acc: 0.18 - ETA: 13:19 - loss: 4.4210 - acc: 0.15 - ETA: 13:11 - loss: 4.4042 - acc: 0.15 - ETA: 12:57 - loss: 4.4043 - acc: 0.15 - ETA: 13:03 - loss: 4.4121 - acc: 0.13 - ETA: 12:51 - loss: 4.4035 - acc: 0.14 - ETA: 12:38 - loss: 4.4106 - acc: 0.14 - ETA: 12:28 - loss: 4.4115 - acc: 0.14 - ETA: 12:17 - loss: 4.4200 - acc: 0.14 - ETA: 12:05 - loss: 4.4159 - acc: 0.14 - ETA: 11:55 - loss: 4.4101 - acc: 0.14 - ETA: 11:44 - loss: 4.4124 - acc: 0.13 - ETA: 11:34 - loss: 4.4085 - acc: 0.14 - ETA: 11:24 - loss: 4.4116 - acc: 0.13 - ETA: 11:13 - loss: 4.4068 - acc: 0.14 - ETA: 11:03 - loss: 4.4128 - acc: 0.14 - ETA: 10:19 - loss: 4.4507 - acc: 0.13 - ETA: 10:12 - loss: 4.4539 - acc: 0.13 - ETA: 10:03 - loss: 4.4505 - acc: 0.13 - ETA: 9:55 - loss: 4.4479 - acc: 0.1339 - ETA: 9:47 - loss: 4.4498 - acc: 0.132 - ETA: 9:38 - loss: 4.4536 - acc: 0.130 - ETA: 9:30 - loss: 4.4513 - acc: 0.131 

83/83 [==============================] - ETA: 13:58 - loss: 4.1933 - acc: 0.21 - ETA: 13:33 - loss: 4.3019 - acc: 0.12 - ETA: 13:28 - loss: 4.2898 - acc: 0.14 - ETA: 13:12 - loss: 4.2980 - acc: 0.13 - ETA: 13:04 - loss: 4.2782 - acc: 0.15 - ETA: 12:57 - loss: 4.2627 - acc: 0.15 - ETA: 12:44 - loss: 4.2609 - acc: 0.16 - ETA: 12:47 - loss: 4.2612 - acc: 0.16 - ETA: 12:38 - loss: 4.2605 - acc: 0.16 - ETA: 12:25 - loss: 4.2634 - acc: 0.16 - ETA: 12:16 - loss: 4.2542 - acc: 0.17 - ETA: 12:06 - loss: 4.2541 - acc: 0.17 - ETA: 11:55 - loss: 4.2507 - acc: 0.18 - ETA: 11:45 - loss: 4.2466 - acc: 0.18 - ETA: 11:34 - loss: 4.2360 - acc: 0.19 - ETA: 11:24 - loss: 4.2417 - acc: 0.19 - ETA: 11:14 - loss: 4.2452 - acc: 0.19 - ETA: 11:03 - loss: 4.2442 - acc: 0.19 - ETA: 10:53 - loss: 4.2350 - acc: 0.19 - ETA: 10:43 - loss: 4.2443 - acc: 0.19 - ETA: 10:32 - loss: 4.2401 - acc: 0.19 - ETA: 10:22 - loss: 4.2379 - acc: 0.19 - ETA: 10:12 - loss: 4.2394 - acc: 0.19 - ETA: 10:01 - loss: 4.2364 - acc: 0.19 -

...

83/83 [==============================] - ETA: 15:26 - loss: 2.0967 - acc: 0.65 - ETA: 15:17 - loss: 1.9911 - acc: 0.67 - ETA: 14:59 - loss: 2.1528 - acc: 0.62 - ETA: 15:03 - loss: 2.1655 - acc: 0.62 - ETA: 14:49 - loss: 2.1454 - acc: 0.65 - ETA: 14:35 - loss: 2.1614 - acc: 0.65 - ETA: 14:21 - loss: 2.1674 - acc: 0.65 - ETA: 14:09 - loss: 2.1932 - acc: 0.63 - ETA: 13:56 - loss: 2.2259 - acc: 0.62 - ETA: 13:41 - loss: 2.2477 - acc: 0.62 - ETA: 13:29 - loss: 2.2391 - acc: 0.61 - ETA: 13:18 - loss: 2.2356 - acc: 0.61 - ETA: 13:05 - loss: 2.2471 - acc: 0.60 - ETA: 12:54 - loss: 2.2451 - acc: 0.61 - ETA: 12:42 - loss: 2.2235 - acc: 0.62 - ETA: 12:31 - loss: 2.2281 - acc: 0.62 - ETA: 12:19 - loss: 2.2173 - acc: 0.62 - ETA: 12:08 - loss: 2.2021 - acc: 0.63 - ETA: 11:57 - loss: 2.1937 - acc: 0.64 - ETA: 11:45 - loss: 2.2020 - acc: 0.63 - ETA: 11:34 - loss: 2.2003 - acc: 0.63 - ETA: 11:23 - loss: 2.2054 - acc: 0.63 - ETA: 11:13 - loss: 2.2070 - acc: 0.63 - ETA: 11:02 - loss: 2.1992 - acc: 0.63 -

83/83 [==============================] - ETA: 14:21 - loss: 1.9838 - acc: 0.65 - ETA: 14:12 - loss: 1.9511 - acc: 0.65 - ETA: 13:51 - loss: 1.9133 - acc: 0.66 - ETA: 13:41 - loss: 1.8726 - acc: 0.69 - ETA: 13:32 - loss: 1.8472 - acc: 0.70 - ETA: 13:18 - loss: 1.8465 - acc: 0.70 - ETA: 13:07 - loss: 1.8712 - acc: 0.70 - ETA: 12:57 - loss: 1.9436 - acc: 0.65 - ETA: 12:44 - loss: 1.9333 - acc: 0.66 - ETA: 12:34 - loss: 1.9601 - acc: 0.65 - ETA: 12:25 - loss: 1.9858 - acc: 0.65 - ETA: 12:13 - loss: 1.9953 - acc: 0.64 - ETA: 12:03 - loss: 1.9911 - acc: 0.64 - ETA: 11:54 - loss: 1.9925 - acc: 0.64 - ETA: 11:43 - loss: 2.0095 - acc: 0.64 - ETA: 11:33 - loss: 2.0027 - acc: 0.64 - ETA: 11:21 - loss: 1.9915 - acc: 0.65 - ETA: 11:11 - loss: 2.0011 - acc: 0.65 - ETA: 11:01 - loss: 1.9907 - acc: 0.65 - ETA: 10:50 - loss: 1.9968 - acc: 0.64 - ETA: 10:40 - loss: 2.0059 - acc: 0.64 - ETA: 10:30 - loss: 2.0009 - acc: 0.65 - ETA: 10:20 - loss: 2.0097 - acc: 0.65 - ETA: 10:11 - loss: 2.0036 - acc: 0.65 -

83/83 [==============================] - ETA: 14:20 - loss: 2.2482 - acc: 0.68 - ETA: 13:49 - loss: 2.0931 - acc: 0.68 - ETA: 13:41 - loss: 2.0094 - acc: 0.69 - ETA: 13:26 - loss: 2.0095 - acc: 0.70 - ETA: 13:13 - loss: 1.9775 - acc: 0.71 - ETA: 13:04 - loss: 1.9372 - acc: 0.72 - ETA: 12:50 - loss: 1.9029 - acc: 0.73 - ETA: 12:41 - loss: 1.9239 - acc: 0.73 - ETA: 12:31 - loss: 1.9034 - acc: 0.73 - ETA: 12:18 - loss: 1.9183 - acc: 0.72 - ETA: 12:09 - loss: 1.9463 - acc: 0.71 - ETA: 11:59 - loss: 1.9657 - acc: 0.70 - ETA: 11:48 - loss: 1.9506 - acc: 0.70 - ETA: 11:39 - loss: 1.9639 - acc: 0.69 - ETA: 11:27 - loss: 1.9585 - acc: 0.69 - ETA: 11:17 - loss: 1.9540 - acc: 0.69 - ETA: 11:08 - loss: 1.9488 - acc: 0.69 - ETA: 10:57 - loss: 1.9567 - acc: 0.69 - ETA: 10:47 - loss: 1.9604 - acc: 0.68 - ETA: 10:38 - loss: 1.9682 - acc: 0.67 - ETA: 10:27 - loss: 1.9773 - acc: 0.67 - ETA: 10:17 - loss: 1.9778 - acc: 0.67 - ETA: 10:06 - loss: 1.9674 - acc: 0.67 - ETA: 9:56 - loss: 1.9646 - acc: 0.6758 

83/83 [==============================] - ETA: 13:27 - loss: 1.8025 - acc: 0.84 - ETA: 13:35 - loss: 1.6899 - acc: 0.81 - ETA: 13:32 - loss: 1.7519 - acc: 0.78 - ETA: 13:30 - loss: 1.7955 - acc: 0.76 - ETA: 13:18 - loss: 1.8315 - acc: 0.73 - ETA: 13:04 - loss: 1.8094 - acc: 0.73 - ETA: 12:53 - loss: 1.7852 - acc: 0.74 - ETA: 12:45 - loss: 1.7977 - acc: 0.72 - ETA: 12:33 - loss: 1.7952 - acc: 0.72 - ETA: 12:30 - loss: 1.8160 - acc: 0.71 - ETA: 12:22 - loss: 1.8296 - acc: 0.71 - ETA: 12:10 - loss: 1.8221 - acc: 0.71 - ETA: 11:59 - loss: 1.8108 - acc: 0.71 - ETA: 11:47 - loss: 1.7970 - acc: 0.72 - ETA: 11:35 - loss: 1.8108 - acc: 0.72 - ETA: 11:25 - loss: 1.8063 - acc: 0.72 - ETA: 11:13 - loss: 1.8230 - acc: 0.71 - ETA: 11:03 - loss: 1.8137 - acc: 0.71 - ETA: 10:53 - loss: 1.8182 - acc: 0.71 - ETA: 10:41 - loss: 1.8091 - acc: 0.71 - ETA: 10:31 - loss: 1.8092 - acc: 0.70 - ETA: 10:20 - loss: 1.8112 - acc: 0.70 - ETA: 10:09 - loss: 1.8082 - acc: 0.70 - ETA: 10:00 - loss: 1.8247 - acc: 0.69 -

KeyboardInterrupt: 

In [33]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [34]:
preds = model.predict_generator(test_generator, verbose=1)

28/28 [==============================] - ETA: 26:0 - ETA: 15:1 - ETA: 11:3 - ETA: 9:4 - ETA: 8: - ETA: 7: - ETA: 6: - ETA: 6: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 56s - ETA: 42 - ETA: 27 - ETA: 13 - 378s 13s/step


In [35]:
preds

array([[2.1688983e-01, 1.1065563e-03, 1.7804666e-03, ..., 5.1173574e-04,
        1.3050667e-03, 1.2941773e-01],
       [7.7352702e-04, 8.1253098e-03, 2.3946064e-03, ..., 1.2656648e-02,
        1.6453847e-02, 9.2768372e-04],
       [1.4433657e-03, 5.7679401e-03, 5.5632456e-03, ..., 6.3613744e-04,
        4.2795641e-03, 1.2343236e-03],
       ...,
       [8.0189138e-04, 1.5122019e-03, 8.9397834e-04, ..., 4.1262107e-04,
        5.3670886e-04, 3.3756447e-04],
       [4.7712920e-06, 7.2116440e-05, 5.8521346e-06, ..., 3.4691486e-05,
        2.4634764e-05, 7.1759655e-06],
       [1.8319199e-03, 6.3747604e-04, 2.0584937e-04, ..., 6.8397560e-05,
        2.6196230e-04, 7.2548348e-05]], dtype=float32)

In [37]:
p=model.predict_generator(test_generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

In [49]:
p.class_indices.keys()
# test_generator.class_indices.keys()

AttributeError: 'numpy.ndarray' object has no attribute 'class_indices'

In [52]:
y_classes = test_generator.argmax(axis=-1)

AttributeError: 'DirectoryIterator' object has no attribute 'argmax'